필요한 라이브러리를 불러온다

In [11]:
import tensorflow as tf
import pickle
import numpy as np
import os
from datetime import datetime
import argparse
import urllib.request
import tarfile
import sys
from pprint import pprint

cifar-10의 데이터를 사용할 것이며, 미리 분류해야할 label의 개수와 웹상에서 cifar-10데이터를 불러오기 위한 url주소를 미리 변수로 지정해 놓음

In [12]:
NCLASS=10
DATA_URL="http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

data_url을 통해서 cifar-10을 다운받는다.
또한 train과 test 폴더를 각각 만들어, 해당되는 data를 각각의  폴더로 이동시킨다.


함수 argument:  
-model_dir : final destination directory of tarfile

In [45]:
def maybe_download_or_extract(model_dir):
    dest_dir=model_dir
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    filename=DATA_URL.split('/')[-1]
    filepath=os.path.join(dest_dir, filename)
    
    if not os.path.exists(filepath):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\r>>Downloading %s %.1f%%'
                %(filename, float(count*block_size)/float(total_size)*100))
            sys.stdout.flush()

        filepath, _=urllib.request.urlretrieve(DATA_URL, filename, _progress)
        statinfo=os.stat(filepath)
        print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
    tarfile.open(filepath, 'r:gz').extractall(model_dir)
    members=tarfile.open(filepath, 'r:gz').getnames()
    sub_dir=os.path.join(model_dir,members[0])
    # print(sub_dir)
    sub_filenames=[os.path.join(sub_dir, filename) for filename in os.listdir(sub_dir)]
    # pprint(sub_filenames)
    #make directory of train data and test data
    train_dir=os.path.join(model_dir, 'train')
    test_dir=os.path.join(model_dir, 'test')

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    train_file_glob=os.path.join(sub_dir, 'data_batch_*')
    train_lists=tf.gfile.Glob(train_file_glob)
    
    for old_train_path in train_lists:
        new_train_path=os.path.join(train_dir, os.path.basename(old_train_path))
        if not os.path.exists(new_train_path):
            tf.gfile.Copy(old_train_path, new_train_path)

    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    test_file_glob=os.path.join(sub_dir, 'test_batch')

    old_test_path=test_file_glob
    new_test_path=os.path.join(test_dir, os.path.basename(new_train_path))
    if not os.path.exists(new_test_path):
        tf.gfile.Copy(old_test_path, new_test_path)

    print('Successfully forming the directory of train and test')

    return train_dir, test_dir
    

웹상에서 다운받은 cifar-10데이터의 라벨을 one-hot encoding을 해준다.

함수 argument:
- pre_label : label basically provided by cifar-10. label that should be one-hot encded

- indices : numpy ndarray. np.array([0,1,2,3,4,5,6,7,8,9]). the number of elements in indices should be equal to the number of classes.

In [14]:
def one_hot_label(pre_label, indices):
	one_label=np.zeros((1,NCLASS))
	index=np.where(indices==pre_label)[0][0]
	one_label[:,index]=1.0
	return one_label

웹상에서 다운받은 cifar-10데이터는 pickle형태로 되어 있다. 따라서 이를 binary파일을 numpy nparray형태로 불러와야한다.

함수 argument:

- batch_path : each individually data_batch pickle file to be unpickled

In [15]:
def load_batch_data(batch_path):
	with open(batch_path, 'rb') as fo:
		batch_data=pickle.load(fo, encoding='bytes')
	data=batch_data[b'data']
	labels=batch_data[b'labels']

	indices=np.arange(NCLASS)
	one_labels=np.zeros([data.shape[0], NCLASS])
	for i,label in enumerate(labels):
		one_label=one_hot_label(label, indices)
		one_labels[i]=one_label

	return data, one_labels

웹상에서 다운 받은 train data와 test data를 모두 다운 받는다.

- 원래는 모든 train data와 test data를 다 받기 위해서 만든 함수이다. 그러나, 실습을 위해서 일부만 다운받는걸로 한다. 

- for batch_patch in batch_paths[:2]:
  --> 원래는 train_batch 파일이 총 5개인데, 이중에서 2개만 불러오게끔 해놓은 것이다. 전체 데이터를 다 불러오고 싶으면 batch_paths[:2]에서 [:2]를 없애주면 된다.
  
함수 argument:
- batch_dir : the upper directory of test and train directory

In [33]:
def load_data(batch_dir):

	batch_lists=os.listdir(batch_dir)
	batch_paths=[os.path.join(batch_dir, batch_list) for batch_list in batch_lists]
	
	if len(batch_paths) > 1:
		full_data=[]
		full_labels=[]
		for batch_path in batch_paths[:2]: ##<-- 전체 데이터를 다 학습시키고 싶으면 [:2]를 제거하면됨!
			data, one_labels=load_batch_data(batch_path)
			full_data.append(data)
			full_labels.append(one_labels)

		full_data=np.asarray(full_data).reshape(-1, 32*32*3)
		full_labels=np.asarray(full_labels).reshape(-1, 10)

		return full_data, full_labels
	elif len(batch_paths) == 1:
		data, labels=load_batch_data(batch_paths[0])
		return data, labels

training시에 graph에 feed할 batch를 구성해준다. 

함수 argument:
- data : train or test data
- one_labels : one-hot encded labels
- train_batch_size : hyperparameter, batch size.

In [19]:
def next_batch(data, one_labels, train_batch_size):	

	index_in_batch=np.random.choice(data.shape[0], size=train_batch_size, replace=False)
	X_batch=data[index_in_batch]
	Y_batch=one_labels[index_in_batch]
	return X_batch, Y_batch

function of convolution, relu, and local response normalization, which is the partial layer of alexnet.
		alexnet consists of 5 convolutional layers and 3 fully connected layers. And also, 5 convolutional layers
		consist of 2 local response layers and 3 non-local response layers. This formulates the local-response layer.
        
함수 argument:

- inputs : input of function
- filter_size : size of receptive field
- num_filters : the number of filter
- prefix - the name of this function
- stride : size of stride
- regularize : if True, weights ar regularized with L2-norm. if False, weights are not regularized.

In [21]:
def conv2d_with_norm(inputs, filter_size, num_filters, prefix, stride=1, padding='SAME', regularize=True):
	he_init=tf.contrib.layers.variance_scaling_initializer() # he initialization
	if regularize==True:
		w_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01)
	elif regularize==False:
		w_regularizer=None

	strides=(stride, stride)

	conv=tf.layers.conv2d(inputs, num_filters, filter_size, strides, padding, activation=tf.nn.relu, 
							kernel_initializer=he_init, 
							kernel_regularizer=w_regularizer, 
							name=prefix+'_conv')
	local_norm=tf.nn.local_response_normalization(conv, name=prefix+'_lrn')
	return local_norm

	this consists of non-local response layer. the remainder intrunction is same as the function right above. 
	Please refer to that function.

In [24]:
def conv2d(inputs, filter_size, num_filters, prefix, stride=1, padding='SAME', regularize=True):
	he_init=tf.contrib.layers.variance_scaling_initializer()
	if regularize==True:
		w_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01)
	elif regularize==False:
		w_regularizer=None

	strides=(stride, stride)
	conv=tf.layers.conv2d(inputs, num_filters, filter_size, strides, padding, activation=tf.nn.relu,
							kernel_initializer=he_init, 
							kernel_regularizer=w_regularizer,
							name=prefix+'_conv')
	return conv

maxpooling layer.

함수 argument:
- inputs : input of maxpooling layer
- pool_size : receptive field of pooling layer
- strides : stride
- prefix : name of this maxpooling layer
- padding : if 'SAME', output tensor has the same size of the input tensor. if 'valid', its not.

In [26]:
def maxpool(inputs, pool_size, strides, prefix, padding='SAME'):
	maxpool=tf.layers.max_pooling2d(inputs, pool_size, strides, padding, name=prefix+'_maxpool')
	return maxpool


This is the alexNet model part. The most of the construction follows the model suggested by the alexnet paper.
However, instead of constructing the 2 local response layers, add an one more local response layer. 
In conclusion, the overall model consists of 3 local response layers and 2 non-local response layers( both are 
convolutional layers) and the rest of 2 layers are fully-connected layers.

In [30]:
def AlexNet():
	X=tf.placeholder(dtype=tf.float32, shape=[None, 32*32*3], name='X') #placeholder if input data
	X_reshaped=tf.reshape(X, shape=[-1, 32, 32, 3])
	keep_prob=0.7 # drop out probability in fully-connected layer

	with tf.name_scope('conv1') as scope:
		conv_n_norm1=conv2d_with_norm(X_reshaped, 5, 64, 'conv1', stride=1)
		pool1=maxpool(conv_n_norm1,3, 2,'conv1')

	with tf.name_scope('conv2') as scope:
		conv_n_norm2=conv2d_with_norm(pool1, 5, 64, 'conv2', stride=1)
		pool2=maxpool(conv_n_norm2, 3, 2, 'conv2')

	with tf.name_scope('conv3') as scope:
		conv3=conv2d(pool2, 3, 128, 'conv3')

	with tf.name_scope('conv4') as scope:
		conv4=conv2d(conv3, 3, 128,'conv4')
	with tf.name_scope('conv5') as scope:
		conv_n_norm5=conv2d_with_norm(conv4, 3, 128, 'conv5')
		pool3=maxpool(conv_n_norm5, 3, 2,'conv5')

	_, w, h, chs=pool3.get_shape().as_list()
	pool3_reshaped=tf.reshape(pool3, [-1, w*h*chs])

	with tf.name_scope('fully') as scope:
		flc1=tf.layers.dense(pool3_reshaped, 384, activation=tf.nn.relu, 
							kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), name='flc1')
		flc1=tf.nn.dropout(flc1, keep_prob)

		flc2=tf.layers.dense(flc1, 192, activation=tf.nn.relu, 
							kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), name='flc2')
		flc2=tf.nn.dropout(flc2, keep_prob)

	with tf.name_scope('output') as scope:
		logits=tf.layers.dense(flc2, units=10, name='logits')

	print(logits.get_shape())
	return X, logits

the function of training part which is added at the final graph of alexnet formed by fucntion AlexNet() mentioned above.

In [32]:
def training_op(logits, learning_rate):
	Y=tf.placeholder(dtype=tf.float32, shape=[None, 10], name='Y')

	with tf.name_scope('cross_entropy'):
		cross_entropy=tf.nn.softmax_cross_entropy_with_logits(
			labels=Y, logits=logits)
		cross_entropy_mean=tf.reduce_mean(cross_entropy)
	tf.summary.scalar('cross_entropy', cross_entropy_mean)

	with tf.name_scope('train'):
		optimizer=tf.train.AdamOptimizer(learning_rate)
		train_step=optimizer.minimize(cross_entropy_mean)

	return train_step, cross_entropy_mean, logits, Y

evalutation part와 시각화를 위해 필요한 tensorboard를 불러온다.

In [37]:
def evaluation_op(result_tensor, ground_truth_tensor):
	with tf.name_scope('accuracy'):
		with tf.name_scope('correct_prediction'):
			prediction=tf.argmax(result_tensor, 1)
			correct_prediction=tf.equal(prediction, tf.argmax(ground_truth_tensor, 1))
		with tf.name_scope('accuracy'):
			accuracy=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
	tf.summary.scalar('accuracy', accuracy)

	return  accuracy, prediction

def variable_summaries(var):
	with tf.name_scope('summaries'):
		mean=tf.reduce_mean(var)
		tf.summary.scalar('mean', mean)
		with tf.summary.scalar('stddev'):
			stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
		tf.summary.scalar('stddev', stddev)
		tf.summary.scalar('max', tf.reduce_max(var))
		tf.summary.scalar('min', tf.reduce_mean(var))
		tf.summary.histogram('histogram', var)

## 이제 본격적으로 학습을 시켜보자. 

model_dir는 test와 train 폴더가 생성될 드렉토리이다. maybe_download_or_extract함수를 이용하여 웹상에서 cifar-10데이터를 다운받아보자. 이때, 함수로 return되는 값은 함수가 돌아가면서 생성된 train폴더와 test폴더의 디렉토리이다.

In [46]:
model_dir = '/home/fdalab/Desktop/dl_cv_tensorflow_10weeks-master/week2/AlexNet'

train_dir, test_dir = maybe_download_or_extract(model_dir)

>>Downloading cifar-10-python.tar.gz 100.0%Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.
Successfully forming the directory of train and test


learning rate, training step, batch size, dislpay step등 필요한 변수들을 지정해준다.

In [52]:
learning_rate=0.01
training_steps=50
batch_size=256
display_step=5

train data와 test data를 불러온다.

In [48]:
train_data, train_one_labels=load_data(train_dir)
test_data, test_one_labels=load_data(test_dir)

위에서 graph 형성 관련된 함수들을 불러와서 graph를 구축한다.

In [49]:
X, logits=AlexNet()

(?, 10)


- X : input에 해당하는 placeholder
- logits : soft max layer의 output.

training_op 함수를 불러와서 training을 위한 graph를 구축해야 한다. 

In [50]:
train_step, cross_entropy_mean, logits, Y=training_op(logits, learning_rate)
accuracy, prediction=evaluation_op(logits, Y)

#### 여기까지가 graph를 구축하는 과정이었다. 이제 session을 열어서, data를 flow시켜주면서 학습을 진행해보자.

summaries_dir : tensorboard에 시각화할 그래프에 관한 logfile이 저장될 디렉토리이다. 

In [53]:
with tf.Session() as sess:
    summaries_dir = './tf_logs'
    
    if tf.gfile.Exists(summaries_dir):
        tf.gfile.DeleteRecursively(summaries_dir)
    tf.gfile.MakeDirs(summaries_dir)
    
    merged=tf.summary.merge_all()
    train_writer=tf.summary.FileWriter(summaries_dir+'/train', tf.get_default_graph())
    test_writer=tf.summary.FileWriter(summaries_dir+'/test')
    
    init=tf.global_variables_initializer()
    ### 모든 파라미터들을 초기화시켜주는 작업은 반드시 들어가야 한다.
    sess.run(init)
    
    step=0
    for epoch in range(training_steps):
        for num in range(int(train_data.shape[0]/batch_size)):
            
            X_batch, Y_batch=next_batch(train_data, train_one_labels, batch_size)
            _, train_summary, train_accuracy, train_loss=sess.run([train_step, merged, accuracy,
                                                                  cross_entropy_mean],
                                                                 feed_dict={X:X_batch, Y:Y_batch})
            train_writer.add_summary(train_summary, step)
            
            X_test_batch, Y_test_batch=next_batch(test_data, test_one_labels, batch_size)
            test_summary, test_accuracy=sess.run([merged, accuracy], feed_dict={X:X_test_batch, Y:Y_test_batch})
            test_writer.add_summary(test_summary, step)
            
            step+=1
            
            if step%display_step==0:
                print('Epoch %d: Step %d: Train accuracy=%1.f%% Train loss=%1.3f' %(epoch, step, train_accuracy*100, train_loss))
                print('Epoch %d: Step %d: Test accuracy=%1.f%%' %(epoch, step, test_accuracy*100))

print('optimization is finished!')

Epoch 0: Step 5: Train accuracy=9% Train loss=3.089
Epoch 0: Step 5: Test accuracy=9%
Epoch 0: Step 10: Train accuracy=11% Train loss=2.374
Epoch 0: Step 10: Test accuracy=10%
Epoch 0: Step 15: Train accuracy=14% Train loss=2.299
Epoch 0: Step 15: Test accuracy=9%
Epoch 0: Step 20: Train accuracy=11% Train loss=2.303
Epoch 0: Step 20: Test accuracy=9%
Epoch 0: Step 25: Train accuracy=6% Train loss=2.329
Epoch 0: Step 25: Test accuracy=10%
Epoch 0: Step 30: Train accuracy=11% Train loss=2.302
Epoch 0: Step 30: Test accuracy=8%
Epoch 0: Step 35: Train accuracy=13% Train loss=2.300
Epoch 0: Step 35: Test accuracy=8%
Epoch 0: Step 40: Train accuracy=6% Train loss=2.307
Epoch 0: Step 40: Test accuracy=10%
Epoch 0: Step 45: Train accuracy=7% Train loss=2.304
Epoch 0: Step 45: Test accuracy=10%
Epoch 0: Step 50: Train accuracy=7% Train loss=2.305
Epoch 0: Step 50: Test accuracy=7%
Epoch 0: Step 55: Train accuracy=15% Train loss=2.300
Epoch 0: Step 55: Test accuracy=7%
Epoch 0: Step 60: Train 

Epoch 5: Step 460: Train accuracy=10% Train loss=2.305
Epoch 5: Step 460: Test accuracy=12%
Epoch 5: Step 465: Train accuracy=12% Train loss=2.300
Epoch 5: Step 465: Test accuracy=8%
Epoch 6: Step 470: Train accuracy=9% Train loss=2.307
Epoch 6: Step 470: Test accuracy=8%
Epoch 6: Step 475: Train accuracy=12% Train loss=2.304
Epoch 6: Step 475: Test accuracy=14%
Epoch 6: Step 480: Train accuracy=10% Train loss=2.303
Epoch 6: Step 480: Test accuracy=13%
Epoch 6: Step 485: Train accuracy=9% Train loss=2.305
Epoch 6: Step 485: Test accuracy=6%
Epoch 6: Step 490: Train accuracy=9% Train loss=2.305
Epoch 6: Step 490: Test accuracy=10%
Epoch 6: Step 495: Train accuracy=10% Train loss=2.303
Epoch 6: Step 495: Test accuracy=10%
Epoch 6: Step 500: Train accuracy=7% Train loss=2.301
Epoch 6: Step 500: Test accuracy=11%
Epoch 6: Step 505: Train accuracy=11% Train loss=2.304
Epoch 6: Step 505: Test accuracy=11%
Epoch 6: Step 510: Train accuracy=9% Train loss=2.302
Epoch 6: Step 510: Test accuracy=

Epoch 11: Step 910: Train accuracy=8% Train loss=2.306
Epoch 11: Step 910: Test accuracy=8%
Epoch 11: Step 915: Train accuracy=9% Train loss=2.302
Epoch 11: Step 915: Test accuracy=11%
Epoch 11: Step 920: Train accuracy=9% Train loss=2.303
Epoch 11: Step 920: Test accuracy=14%
Epoch 11: Step 925: Train accuracy=13% Train loss=2.303
Epoch 11: Step 925: Test accuracy=13%
Epoch 11: Step 930: Train accuracy=13% Train loss=2.303
Epoch 11: Step 930: Test accuracy=8%
Epoch 11: Step 935: Train accuracy=9% Train loss=2.304
Epoch 11: Step 935: Test accuracy=12%
Epoch 12: Step 940: Train accuracy=11% Train loss=2.304
Epoch 12: Step 940: Test accuracy=14%
Epoch 12: Step 945: Train accuracy=11% Train loss=2.303
Epoch 12: Step 945: Test accuracy=11%
Epoch 12: Step 950: Train accuracy=8% Train loss=2.303
Epoch 12: Step 950: Test accuracy=11%
Epoch 12: Step 955: Train accuracy=12% Train loss=2.301
Epoch 12: Step 955: Test accuracy=8%
Epoch 12: Step 960: Train accuracy=12% Train loss=2.300
Epoch 12: St

Epoch 17: Step 1345: Train accuracy=9% Train loss=2.301
Epoch 17: Step 1345: Test accuracy=7%
Epoch 17: Step 1350: Train accuracy=11% Train loss=2.301
Epoch 17: Step 1350: Test accuracy=11%
Epoch 17: Step 1355: Train accuracy=9% Train loss=2.302
Epoch 17: Step 1355: Test accuracy=12%
Epoch 17: Step 1360: Train accuracy=12% Train loss=2.303
Epoch 17: Step 1360: Test accuracy=7%
Epoch 17: Step 1365: Train accuracy=11% Train loss=2.302
Epoch 17: Step 1365: Test accuracy=8%
Epoch 17: Step 1370: Train accuracy=9% Train loss=2.305
Epoch 17: Step 1370: Test accuracy=10%
Epoch 17: Step 1375: Train accuracy=8% Train loss=2.304
Epoch 17: Step 1375: Test accuracy=9%
Epoch 17: Step 1380: Train accuracy=12% Train loss=2.304
Epoch 17: Step 1380: Test accuracy=11%
Epoch 17: Step 1385: Train accuracy=11% Train loss=2.303
Epoch 17: Step 1385: Test accuracy=11%
Epoch 17: Step 1390: Train accuracy=11% Train loss=2.303
Epoch 17: Step 1390: Test accuracy=14%
Epoch 17: Step 1395: Train accuracy=9% Train los

Epoch 22: Step 1780: Train accuracy=9% Train loss=2.304
Epoch 22: Step 1780: Test accuracy=14%
Epoch 22: Step 1785: Train accuracy=7% Train loss=2.301
Epoch 22: Step 1785: Test accuracy=12%
Epoch 22: Step 1790: Train accuracy=11% Train loss=2.304
Epoch 22: Step 1790: Test accuracy=10%
Epoch 23: Step 1795: Train accuracy=14% Train loss=2.305
Epoch 23: Step 1795: Test accuracy=10%
Epoch 23: Step 1800: Train accuracy=9% Train loss=2.301
Epoch 23: Step 1800: Test accuracy=7%
Epoch 23: Step 1805: Train accuracy=11% Train loss=2.301
Epoch 23: Step 1805: Test accuracy=7%
Epoch 23: Step 1810: Train accuracy=10% Train loss=2.301
Epoch 23: Step 1810: Test accuracy=12%
Epoch 23: Step 1815: Train accuracy=9% Train loss=2.303
Epoch 23: Step 1815: Test accuracy=9%
Epoch 23: Step 1820: Train accuracy=9% Train loss=2.305
Epoch 23: Step 1820: Test accuracy=11%
Epoch 23: Step 1825: Train accuracy=11% Train loss=2.303
Epoch 23: Step 1825: Test accuracy=7%
Epoch 23: Step 1830: Train accuracy=9% Train loss

Epoch 28: Step 2215: Train accuracy=14% Train loss=2.301
Epoch 28: Step 2215: Test accuracy=6%
Epoch 28: Step 2220: Train accuracy=8% Train loss=2.304
Epoch 28: Step 2220: Test accuracy=7%
Epoch 28: Step 2225: Train accuracy=11% Train loss=2.303
Epoch 28: Step 2225: Test accuracy=9%
Epoch 28: Step 2230: Train accuracy=9% Train loss=2.304
Epoch 28: Step 2230: Test accuracy=9%
Epoch 28: Step 2235: Train accuracy=10% Train loss=2.303
Epoch 28: Step 2235: Test accuracy=9%
Epoch 28: Step 2240: Train accuracy=10% Train loss=2.304
Epoch 28: Step 2240: Test accuracy=11%
Epoch 28: Step 2245: Train accuracy=7% Train loss=2.305
Epoch 28: Step 2245: Test accuracy=12%
Epoch 28: Step 2250: Train accuracy=10% Train loss=2.301
Epoch 28: Step 2250: Test accuracy=9%
Epoch 28: Step 2255: Train accuracy=11% Train loss=2.303
Epoch 28: Step 2255: Test accuracy=8%
Epoch 28: Step 2260: Train accuracy=8% Train loss=2.303
Epoch 28: Step 2260: Test accuracy=15%
Epoch 29: Step 2265: Train accuracy=9% Train loss=2

Epoch 33: Step 2650: Train accuracy=13% Train loss=2.300
Epoch 33: Step 2650: Test accuracy=9%
Epoch 34: Step 2655: Train accuracy=13% Train loss=2.301
Epoch 34: Step 2655: Test accuracy=11%
Epoch 34: Step 2660: Train accuracy=11% Train loss=2.302
Epoch 34: Step 2660: Test accuracy=7%
Epoch 34: Step 2665: Train accuracy=9% Train loss=2.304
Epoch 34: Step 2665: Test accuracy=11%
Epoch 34: Step 2670: Train accuracy=11% Train loss=2.304
Epoch 34: Step 2670: Test accuracy=11%
Epoch 34: Step 2675: Train accuracy=11% Train loss=2.302
Epoch 34: Step 2675: Test accuracy=4%
Epoch 34: Step 2680: Train accuracy=6% Train loss=2.305
Epoch 34: Step 2680: Test accuracy=12%
Epoch 34: Step 2685: Train accuracy=10% Train loss=2.304
Epoch 34: Step 2685: Test accuracy=9%
Epoch 34: Step 2690: Train accuracy=9% Train loss=2.302
Epoch 34: Step 2690: Test accuracy=11%
Epoch 34: Step 2695: Train accuracy=10% Train loss=2.304
Epoch 34: Step 2695: Test accuracy=10%
Epoch 34: Step 2700: Train accuracy=11% Train l

Epoch 39: Step 3085: Train accuracy=12% Train loss=2.302
Epoch 39: Step 3085: Test accuracy=11%
Epoch 39: Step 3090: Train accuracy=8% Train loss=2.304
Epoch 39: Step 3090: Test accuracy=8%
Epoch 39: Step 3095: Train accuracy=10% Train loss=2.300
Epoch 39: Step 3095: Test accuracy=7%
Epoch 39: Step 3100: Train accuracy=12% Train loss=2.302
Epoch 39: Step 3100: Test accuracy=13%
Epoch 39: Step 3105: Train accuracy=12% Train loss=2.301
Epoch 39: Step 3105: Test accuracy=13%
Epoch 39: Step 3110: Train accuracy=11% Train loss=2.302
Epoch 39: Step 3110: Test accuracy=12%
Epoch 39: Step 3115: Train accuracy=9% Train loss=2.306
Epoch 39: Step 3115: Test accuracy=7%
Epoch 39: Step 3120: Train accuracy=9% Train loss=2.301
Epoch 39: Step 3120: Test accuracy=12%
Epoch 40: Step 3125: Train accuracy=11% Train loss=2.303
Epoch 40: Step 3125: Test accuracy=13%
Epoch 40: Step 3130: Train accuracy=12% Train loss=2.302
Epoch 40: Step 3130: Test accuracy=9%
Epoch 40: Step 3135: Train accuracy=8% Train lo

Epoch 45: Step 3520: Train accuracy=12% Train loss=2.301
Epoch 45: Step 3520: Test accuracy=10%
Epoch 45: Step 3525: Train accuracy=12% Train loss=2.301
Epoch 45: Step 3525: Test accuracy=10%
Epoch 45: Step 3530: Train accuracy=12% Train loss=2.302
Epoch 45: Step 3530: Test accuracy=7%
Epoch 45: Step 3535: Train accuracy=6% Train loss=2.304
Epoch 45: Step 3535: Test accuracy=9%
Epoch 45: Step 3540: Train accuracy=9% Train loss=2.306
Epoch 45: Step 3540: Test accuracy=10%
Epoch 45: Step 3545: Train accuracy=8% Train loss=2.302
Epoch 45: Step 3545: Test accuracy=8%
Epoch 45: Step 3550: Train accuracy=12% Train loss=2.303
Epoch 45: Step 3550: Test accuracy=11%
Epoch 45: Step 3555: Train accuracy=11% Train loss=2.301
Epoch 45: Step 3555: Test accuracy=9%
Epoch 45: Step 3560: Train accuracy=8% Train loss=2.304
Epoch 45: Step 3560: Test accuracy=9%
Epoch 45: Step 3565: Train accuracy=8% Train loss=2.305
Epoch 45: Step 3565: Test accuracy=9%
Epoch 45: Step 3570: Train accuracy=8% Train loss=2

#### tensorboard 실행 방법

terminal이나 cmd창을 켜서, model_dir로 간다.

![title](./img/cmd창.png)


 가상환경으로 tensorflow를 깔면 가상환경을 킨 뒤,  tensorboard --log_dir ./tf_logs라 입력한다.

![title](./img/cmd4.png)

enter를 치면 다음과 같이 화면이 뜬다.

![title](./img/cmd5.png)

위의 주소를 복사한 후 익스플로어나 크롬을 켜서 해당 페이지로 들어가면 tensorboard 페이지가 뜬다.

![title](./img/screen1.png)

tensorboard 위의 주황색 작업표시줄에 graph를 클릭하면, 구축한 모델의 graph가 나온다.

![title](./img/graph1.png)